In [1]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import h5py
import time

# Test NetCDF4 against H5py

In [2]:
n_ics = 2
step = 32
pert = 50
var = 5
lat = 160
lon = 230

## NetCDF4

In [3]:
#Create Dataset
root = Dataset("test.nc", "a", format = "NETCDF4")

In [4]:
# Create dimensions
root.createDimension("ic", n_ics)
root.createDimension("step", step)
root.createDimension("pert", pert)
root.createDimension("var", var)
root.createDimension("lat", lat)
root.createDimension("long", lon)

<class 'netCDF4._netCDF4.Dimension'>: name = 'long', size = 230

In [5]:
# Create variables
fourcast = root.createVariable("fourcast", "f4", ("ic", "pert", "step", "var", "lat", "long"), compression = "zlib")
ground_truth = root.createVariable("ground_truth", "f4", ("ic", "step", "var", "lat", "long"), compression = "zlib")

In [6]:
norm1 = np.random.normal(size = (step, var, lat, lon))
norm2 = np.random.normal(size = (pert, step, var, lat, lon))

#Write file
t1 = time.time()
ground_truth[0] = norm1
fourcast[0] = norm2
t2 = time.time()-t1
root.close()

print("Time  = {}".format(t2))

Time  = 29.05280303955078


## H5py

In [56]:
f = h5py.File("test.h5", 'a')

In [57]:
try:
    dataset_pred = f.create_dataset(name = "predictions", shape = (n_ics, pert, step, var, lat, lon), dtype = np.float32, compression = "gzip", compression_opts=9)
except:
    del f["predictions"]
    dataset_pred = f.create_dataset(name = "predictions", shape = (n_ics, pert, step, var, lat, lon), dtype = np.float32, compression = "gzip", compression_opts=9)

#Create default dataset for real values
try:
    dataset_truth = f.create_dataset(name = "ground_truth", shape = (n_ics, step, var, lat, lon), dtype = np.float32, compression = "gzip", compression_opts=9)
except:
    del f["ground_truth"]
    dataset_truth = f.create_dataset(name = "ground_truth", shape = (n_ics, step, var, lat, lon), dtype = np.float32, compression = "gzip", compression_opts=9)

In [58]:
for i in range(n_ics):
    norm1 = np.random.normal(size = (step, var, lat, lon))
    norm2 = np.random.normal(size = (pert, step, var, lat, lon))
    dataset_truth[i] = norm1
    dataset_pred[i] = norm2
f.close()